In [160]:
import numpy as np 
import torch
import torch.nn as nn 
from dataCollection import getData
import matplotlib.pyplot as plt 
import warnings
warnings.filterwarnings("ignore")

$ut + uux − (0.01/π)uxx = 0, x ∈ [−1, 1], t ∈ [0, 1]$

$u(0, x) = − sin(πx)$,

$u(t, −1) = u(t, 1) = 0$

In [135]:
torch.manual_seed(42)

In [105]:
#Make a neural network for u 
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.fc1 = nn.Linear(2, 16)
        self.fc2 = nn.Linear(16,32)
        self.fc3 = nn.Linear(32,1)
    
    def forward(self,x):
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [71]:
X_star, u_star, lb, ub = getData()

In [72]:
X_star

tensor([[-1.0000,  0.0000],
        [-0.9922,  0.0000],
        [-0.9843,  0.0000],
        ...,
        [ 0.9843,  0.9900],
        [ 0.9922,  0.9900],
        [ 1.0000,  0.9900]])

In [74]:
u_star.flatten()[:, None]

array([[ 1.22464680e-16],
       [ 2.46374492e-02],
       [ 4.92599411e-02],
       ...,
       [-1.19673204e-02],
       [-5.98368729e-03],
       [ 1.12388795e-16]])

In [75]:
lb, ub

(torch.return_types.min(
 values=tensor([-1.,  0.]),
 indices=tensor([0, 0])),
 torch.return_types.max(
 values=tensor([1.0000, 0.9900]),
 indices=tensor([  255, 25344])))

In [156]:
#Making the Physics Informed NN here, look at `physics` to adjust between PINN and Vanilla NN.
class PINN():
    def __init__(self, X, u, lb, ub, physics):
        
        self.lb = torch.tensor(lb).float()
        self.ub = torch.tensor(ub).float()
        self.physics = physics
        
        self.x = torch.tensor(X[:, 0:1], requires_grad=True).float()
        self.t = torch.tensor(X[:, 1:2], requires_grad=True).float()
        self.u = torch.tensor(u).float()
        
        self.network = Network()
        
        self.optimizer = torch.optim.Adam(self.network.parameters(), lr=0.001)
        
    def makeNetwork(self, x,t):
        X = torch.cat([x,t],1)
        return self.network(X)
    
    def residual(self, x, t):
        
        u = self.makeNetwork(x, t)
        u_t = torch.autograd.grad(u, t, grad_outputs=torch.ones_like(u),  create_graph=True)[0]
        u_x = torch.autograd.grad(u, x, grad_outputs=torch.ones_like(u) , create_graph=True)[0]
        u_xx = torch.autograd.grad(u_x, x, grad_outputs = torch.ones_like(u_x) ,create_graph=True)[0]
        
        return u_t + u*u_x - (0.01/np.pi)*u_xx
    
    def lossResidual(self):
        
        u_pred = self.makeNetwork(self.x, self.t)
        residual_pred = self.residual(self.x, self.t)
        loss = torch.mean((self.u - u_pred)**2)
        if self.physics == True:  
            loss += torch.mean(residual_pred**2)
        self.optimizer.zero_grad()
        loss.backward()
        return loss
    
    def train(self, epochs):
        lossTracker = []
        self.network.train()
        for idx in range(epochs):
            u_pred = self.makeNetwork(self.x, self.t)
            residual_pred = self.residual(self.x, self.t)
            loss = torch.mean((self.u - u_pred)**2)
            if self.physics == True:
                loss += torch.mean(residual_pred**2)
            #print(f"The loss at epoch {idx} is {loss.item()}")
            lossTracker.append(loss.item())
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            self.optimizer.step(self.lossResidual)
        return lossTracker
            
    def predict(self): 
        self.network.eval()
        u = self.makeNetwork(self.x, self.t)
        res = self.residual(self.x, self.t)
        return u.detach().numpy(), res.detach().numpy()
        

In [157]:
idx = np.random.choice(X_star.shape[0], 2000, replace=False)
X_u_train = X_star[idx, :]
u_train = u_star.flatten()[:, None][idx,:]

# model = PINN(X_u_train, u_train, lb[0], ub[0])

Keeping `physics` parameter true enables the residual losses to act

In [161]:
model = PINN(X_u_train, u_train, lb[0], ub[0], True)
pinn = model.train(1000)

Here the `physics` parameter is False, and this is just a Vanilla NN.

In [162]:
model = PINN(X_u_train, u_train, lb[0], ub[0], False)
no_pinn = model.train(1000)

In [163]:
import plotly.graph_objects as go

epochs = list(range(len(pinn)))
fig = go.Figure()
fig.add_trace(go.Scatter(x=epochs, y=pinn, mode='lines', name='Physics Informed Neural Network'))
fig.update_layout(
    title='Loss vs. Epochs',
    xaxis=dict(title='Epochs'),
    yaxis=dict(title='Loss'),
    legend=dict(x=0.7, y=1.0),
    margin=dict(l=20, r=20, t=40, b=20),
    hovermode='x unified' 
)
fig.show()

In [164]:
epochs = list(range(len(pinn)))
fig = go.Figure()
fig.add_trace(go.Scatter(x=epochs, y=no_pinn, mode='lines', name='Not Physics Informed Neural Network'))
fig.update_layout(
    title='Loss vs. Epochs',
    xaxis=dict(title='Epochs'),
    yaxis=dict(title='Loss'),
    legend=dict(x=0.7, y=1.0),
    margin=dict(l=20, r=20, t=40, b=20),
    hovermode='x unified' 
)
fig.show()